# Setup Alexnet (Part D)
Transfer learning using layer fc7 labels 

Resize images so they fit to minimum dimensionality required in AlexNet 

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms

# Modify size of images, convert them to tensors and normalize them, hence use transform composition to do so
transform = transforms.Compose([transforms.Resize(224), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_set = CIFAR10(root='data/', train=False, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/


Attach the runtime to CUDA cores for multiprocessing if needed, then update the AlexNet architecture such that layer FC6 outputs 10 class labels to match the dimensionality of CIFAR-10

In [ ]:
device = torch.device('cuda:0')

In [ ]:
import torchvision.models as models
import torch.nn as nn

alexnet_model = models.alexnet(pretrained=True)
alexnet_model.eval()

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
# Perform GPU mapping to improve processing time 
alexnet_model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Due to processing limitations on Google Collab, formulate subset of training data, then create train/test sets from it

In [ ]:
# Form subset range of indices
subset = list(range(0, 10000, 1))

# Load CIFAR-10 data
train_data = CIFAR10(root='data/', train=True, download=True, transform=transform)
subset_train_data = torch.utils.data.Subset(train_data, subset)

# Create train and validation sets based on subset 
train_set, val_set = torch.utils.data.random_split(subset_train_data, [8000, 2000])

# Formulate functions that load data in batches for processing to AlexNet
trainloader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, num_workers=1)
valloader = torch.utils.data.DataLoader(val_set, batch_size=1, shuffle=True, num_workers=1)

Files already downloaded and verified


In [ ]:
# Only require output from 'fc7', hence split AlexNet from there
alexnet_model.classifier = nn.Sequential(*list(alexnet_model.classifier.children())[:-2])
print(alexnet_model.classifier)

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
)


# Grab Features From Layer 'fc7' and Generate Train, Validation Sets

In [ ]:
# Begin evaluation
alexnet_model.eval()
alexnet_model.to(device)

# PyTorch performs batch processing, require arrays to store all values together 

# Formulate arrays to store predicted class label, true label, and associated image respectively for train set
fc7_out_train = []
train_labels = []

# Formulate arrays to store predicted class label, true label, and associated image respectively for test set
fc7_out_val = []
val_labels = []

# Run batch processing through AlexNet using dataloaders
with torch.no_grad():
  for i, data in enumerate(trainloader):
    images, labels = data[0].to(device), data[1].to(device)
    
    # Retrieve from CPU memory as CUDA core storage is unique
    # Get true class output from data
    train_labels.append(labels.cpu().numpy())

    # Get AlexNet output features from 'FC6'
    outputs = alexnet_model(images)
    outputs_np = np.array(outputs.cpu().numpy())
    outputs_np = outputs_np.flatten().tolist()
    fc7_out_train.append(outputs_np)

with torch.no_grad():
  for i, data in enumerate(valloader):
    images, labels = data[0].to(device), data[1].to(device)

    # Retrieve from CPU memory as CUDA core storage is unique
    # Get true class output from data
    val_labels.append(labels.cpu().numpy())

    # Get AlexNet output features from fc7
    outputs = alexnet_model(images)
    outputs_np = np.array(outputs.cpu().numpy())
    outputs_np = outputs_np.flatten().tolist()
    fc7_out_val.append(outputs_np)

In [ ]:
# Retrieve predicted labels from training set
fc7_out_train_np = np.array(fc7_out_train)
print(fc7_out_train_np.shape)

(8000, 4096)


In [ ]:
# Retrieve true labels for training and validation sets of data
train_labels_np = np.array(train_labels)
train_labels_np = train_labels_np.flatten()

val_labels_np = np.array(val_labels)
val_labels_np = val_labels_np.flatten()

print(train_labels_np.shape)
print(val_labels_np.shape)

(8000,)
(2000,)


# Fit Support Vector Machine (SVM) for Classification

Given generated training, validation images, predictions and true labels, can pass data into linear SVM to attempt classification. 

In [ ]:
from sklearn import svm

clf = svm.SVC(kernel='linear')

# Pass each image from train set and the associated predicted class label for fitting
clf.fit(fc7_out_train_np, train_labels_np)

# Perform predictions on the validation set now
pred_val = clf.predict(fc7_out_val)

In [ ]:
from sklearn import metrics

# Check the accuracy score between the validation set true labels and the predicted labels
# Essentially determining if the fc7 layer of AlexNet effectively classified each image
print("Accuracy:", metrics.accuracy_score(val_labels_np, pred_val))

Accuracy: 0.764
